## python machine learning svm, pca, gbm
!pip install scikit-learn 

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [2]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

In [3]:
iris_data = load_iris()

In [4]:
iris_x = iris_data.data
iris_y = iris_data.target

## 붓꽃의 꽃받침 길이, 꽃받침 너비 과 꽃잎의 길이, 꽃잎의 너비 에 대한 값 
type(iris_x), iris_x.shape, iris_x[:10, ]

## class number 0- setosa, 1 - versicolor, 2 - virginica
type(iris_y), iris_y.shape, iris_y[:, ]

In [5]:
# norm 0~1 .. input mapping 

def norm_maxmin(data, out_min=0, out_max=255):
    
    gradient = (out_max - out_min) / (np.max(data) - np.min(data))
    bias = out_min - np.min(data) * gradient
    out = data * gradient + bias

    return out

In [6]:
# raw, col 
for it_col in range(iris_x.shape[1]):
    iris_x[:, it_col] = norm_maxmin(iris_x[:, it_col], 0, 1)

In [7]:
x_train, x_test, y_train, y_test = train_test_split(iris_x, iris_y, test_size=0.1)

x_train.shape, y_train.shape, x_test.shape, y_test.shape

((135, 4), (135,), (15, 4), (15,))

In [8]:
batch_size = 15

In [9]:
x_train = x_train.reshape(-1, batch_size, x_train.shape[1])
y_train = y_train.reshape(-1, batch_size, )

In [10]:
x_train.shape, y_train.shape

((9, 15, 4), (9, 15))

In [11]:
# numpy ary to tensor 
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

x_test = torch.FloatTensor(x_test)
y_test = torch.LongTensor(y_test)

In [12]:
type(x_train), x_train.shape, x_train.dtype, type(y_train), y_train.shape, y_train.dtype

(torch.Tensor,
 torch.Size([9, 15, 4]),
 torch.float32,
 torch.Tensor,
 torch.Size([9, 15]),
 torch.int64)

# model init

In [13]:
class Iris_FCN(nn.Module):
    
    def __init__(self, input_dim, output_dim):
        super(Iris_FCN, self).__init__()
        # 
        self.fc = nn.Linear(input_dim, 16)
        self.relu = nn.ReLU(inplace=True)
        self.fc2 = nn.Linear(16, 8)
        self.relu2 = nn.ReLU(inplace=True)
        self.fc3 = nn.Linear(8, output_dim)
        self.relu3 = nn.ReLU(inplace=True)
    
    def forward(self, x):
        o = self.fc(x)
        o = self.relu(o)
        o = self.fc2(o)
        o = self.relu2(o)
        o = self.fc3(o)
        # o = self.relu3(o)
        
        # softmax
        
        return o

In [14]:
iris_net = Iris_FCN(4, 3)

# from torchsummary import summary

summary(iris_net, input_size=((1, 4)))

In [15]:
# train
criterion = nn.CrossEntropyLoss() 
optimizer = optim.Adam(iris_net.parameters(), lr=0.001)

In [16]:
epochs = 10000
for e in range(epochs):
    for b in range(x_train.shape[0]):
        optimizer.zero_grad()
        
        output = iris_net(x_train[b])
        
        loss = criterion(output, y_train[b])
        loss.backward()
        
        optimizer.step()
    
    if (e+1) % 1000 == 0:
        print(f'Epoch {e+1} / {epochs}, Loss : {loss.item()}')
        # accuracy 
        with torch.no_grad():
            iris_net.eval()

            correct = 0
            total = 0

            outputs = iris_net(x_test)
            pred = torch.argmax(outputs, axis=1)
            total = y_test.size(0)
            correct = (pred == y_test).sum().item()
            print(f'correct { correct } / total {total}')
            print('Accuracy of the model on test data: {} %'.format(100 * correct / total))
        iris_net.train()
        

Epoch 1000 / 10000, Loss : 0.03439458832144737
correct 15 / total 15
Accuracy of the model on test data: 100.0 %
Epoch 2000 / 10000, Loss : 0.02870650216937065
correct 15 / total 15
Accuracy of the model on test data: 100.0 %
Epoch 3000 / 10000, Loss : 0.025904323905706406
correct 15 / total 15
Accuracy of the model on test data: 100.0 %
Epoch 4000 / 10000, Loss : 0.024247998371720314
correct 15 / total 15
Accuracy of the model on test data: 100.0 %
Epoch 5000 / 10000, Loss : 0.023209143429994583
correct 15 / total 15
Accuracy of the model on test data: 100.0 %
Epoch 6000 / 10000, Loss : 0.022868728265166283
correct 15 / total 15
Accuracy of the model on test data: 100.0 %
Epoch 7000 / 10000, Loss : 0.022511309012770653
correct 15 / total 15
Accuracy of the model on test data: 100.0 %
Epoch 8000 / 10000, Loss : 0.022217078134417534
correct 15 / total 15
Accuracy of the model on test data: 100.0 %
Epoch 9000 / 10000, Loss : 0.021986979991197586
correct 15 / total 15
Accuracy of the mode